In [ ]:
import math
import numpy as np
from scipy import signal
import utils
import mediapipe as mp
import cv2

from mediapipe.python.solutions.face_mesh_connections import (
    FACEMESH_FACE_OVAL,
    FACEMESH_LEFT_EYE,
    FACEMESH_RIGHT_EYE,
    FACEMESH_LIPS,
)

mp_face_mesh = mp.solutions.face_mesh

# -------------------------
# Load video
# -------------------------
video_path = "Dataset/Patient_1/Q1_1/vid_crop.avi"
cap = cv2.VideoCapture(video_path)

# -------------------------
# Initialize FaceMesh ONCE
# -------------------------
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)

# -------------------------
# Helper: remove region by convex polygon
# -------------------------
def erase_region(mask, face, pairs, w, h):
    idxs = list(set([i for p in pairs for i in p]))
    pts = np.array([
        [int(face.landmark[i].x * w), int(face.landmark[i].y * h)]
        for i in idxs
    ])
    if len(pts) > 0:
        cv2.fillConvexPoly(mask, pts, 0)


# -------------------------
# MAIN LOOP
# -------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    h, w, _ = frame.shape

    # -------------------------
    # Run MediaPipe
    # -------------------------
    results = face_mesh.process(frame_rgb)
    if not results.multi_face_landmarks:
        print("No face detected.")
        continue

    face = results.multi_face_landmarks[0]

    # -------------------------
    # Create mask
    # -------------------------
    mask = np.zeros((h, w), dtype=np.uint8)

    # ---- FACE OVAL ----
    oval_idxs = list(set([i for pair in FACEMESH_FACE_OVAL for i in pair]))
    oval_points = np.array([
        [int(face.landmark[i].x * w), int(face.landmark[i].y * h)]
        for i in oval_idxs
    ])

    cv2.fillConvexPoly(mask, oval_points, 255)

    # ---- REMOVE EYES ----
    erase_region(mask, face, FACEMESH_LEFT_EYE, w, h)
    erase_region(mask, face, FACEMESH_RIGHT_EYE, w, h)

    # ---- REMOVE LIPS ----
    erase_region(mask, face, FACEMESH_LIPS, w, h)

    # ---- Smooth mask ----
    mask = cv2.GaussianBlur(mask, (21, 21), 0)

    # -------------------------
    # APPLY MASK TO FRAME
    # -------------------------
    masked_frame = np.zeros_like(frame_rgb)
    masked_frame[mask > 0] = frame_rgb[mask > 0]

    cv2.imshow("Skin Mask Only", cv2.cvtColor(masked_frame, cv2.COLOR_RGB2BGR))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
